In [1]:
!pip install mlxtend

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [3]:
# Load the XLSX file using pandas
xlsx_file = pd.read_excel('Online Retail.xlsx')

# Save the dataframe as a CSV file
xlsx_file.to_csv('Online_Retail.csv', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df=pd.read_csv("Online_Retail.csv",encoding='latin1')
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(541909, 8)

In [6]:
df.dtypes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [7]:
df.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Data Cleaning

In [8]:
df['Description']=df['Description'].str.strip()#remove empty spaces

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df.dropna(axis=0 , subset=['InvoiceNo'],inplace=True)#drop rows that dont have invoice number

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df['InvoiceNo']=df['InvoiceNo'].astype('str')# convert invoice no to string


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df=df[~df['InvoiceNo'].str.contains('C')]#drop rows with invoices containing C which means CreditTransaction

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(532621, 8)

after cleaning data we need to consolidate items into 1 transaction per row with each product 1 hot encoded.This notebook contains market basket analysis for France only you can change the name of country according to yourself

In [13]:
df.Country.unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [14]:
#get data for france group them on invoiceNo and Description based on Quantity sum quantity unstack them fill 0 to non value and set index asinvoiceNo
basket=(df[df['Country']=="France"].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
basket.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                          0.0                         0.0   
536852                          0.0                         0.0   
536974                          0.0                         0.0   
537065                          0.0                         0.0   
537463                          0.0                         0.0   

Description  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                                                 
536370                             0.0                              0.0   
536852                             0.0                              0.0   
536974                             0.0                              0.0   
537065                             0.0                              0.0   
537463                             0.0                              0.0   

Description  12 PENCIL SMALL TUBE WOODLAND  \
InvoiceNo                                    
536370                                 0.0   
536852                                 0.0   
536974                                 0.0   
537065                                 0.0   
537463                                 0.0   

Description  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  \
InvoiceNo                                                                       
536370                                       0.0                          0.0   
536852                                       0.0                          0.0   
536974                                       0.0                          0.0   
537065                                       0.0                          0.0   
537463                                       0.0                          0.0   

Description  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  \
InvoiceNo                                                                    
536370                             0.0                                 0.0   
536852                             0.0                                 0.0   
536974                             0.0                                 0.0   
537065                             0.0                                 0.0   
537463                             0.0                                 0.0   

Description  12 PENCILS TALL TUBE WOODLAND  \
InvoiceNo                                    
536370                                 0.0   
536852                                 0.0   
536974                                 0.0   
537065                                 0.0   
537463                                 0.0   

Description  15CM CHRISTMAS GLASS BALL 20 LIGHTS  \
InvoiceNo                                          
536370                                       0.0   
536852                                       0.0   
536974                                       0.0   
537065                                       0.0   
537463                                       0.0   

Description  16 PIECE CUTLERY SET PANTRY DESIGN  \
InvoiceNo                                         
536370                                      0.0   
536852                                      0.0   
536974                                      0.0   
537065                                      0.0   
537463                                      0.0   

Description  18PC WOODEN CUTLERY SET DISPOSABLE  20 DOLLY PEGS RETROSPOT  \
InvoiceNo                                                                  
536370                                      0.0                      0.0   
536852                                      0.0                      0.0   
536974                                      0.0                      0.0   
537065                                      0.0                      0.0   
537463                                      0.0                 

In [16]:
basket.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(392, 1563)

In [17]:
#function converts a value<0  to 0 and value>=1 to 1
def encode_units(x):
 if x<=0:
  return 0
 if x>=1:
  return 1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
#apply function to basket
basket_sets=basket.applymap(encode_units)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
basket_sets.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Description,12 PENCILS TALL TUBE RED RETROSPOT,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,36 PENCILS TUBE RED RETROSPOT,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,60 CAKE CASES DOLLY GIRL DESIGN,ABC TREASURE BOOK BOX,ASSORTED EASTER GIFT TAGS,BASKET OF FLOWERS SEWING KIT,BASKET OF TOADSTOOLS,BLACK MINI TAPE MEASURE,BLOSSOM IMAGES GIFT WRAP SET,BOX OF VINTAGE JIGSAW BLOCKS,BUFFALO BILL TREASURE BOOK BOX,CARD BILLBOARD FONT,CARD CAT AND TREE,CARD CIRCUS PARADE,CARD GINGHAM ROSE,CERAMIC CHERRY CAKE MONEY BANK,CHARLOTTE BAG DOLLY GIRL DESIGN,CHILDREN'S APRON DOLLY GIRL,CHILDRENS APRON SPACEBOY DESIGN,CLOTHES PEGS RETROSPOT PACK 24,CURIOUS IMAGES GIFT WRAP SET,DECORATIVE CATS BATHROOM BOTTLE,DELUXE SEWING KIT,DOCTOR'S BAG SOFT TOY,DOORMAT I LOVE LONDON,EASTER DECORATION EGG BUNNY,EASTER DECORATION SITTING BUNNY,EASTER TIN BUCKET,EASTER TIN BUNNY BOUQUET,EASTER TIN CHICKS IN GARDEN,EASTER TIN KEEPSAKE,"ELEPHANT, BIRTHDAY CARD,",EMERGENCY FIRST AID TIN,ENVELOPE 50 BLOSSOM IMAGES,FAIRY TALE COTTAGE NIGHT LIGHT,"FANCY FONT BIRTHDAY CARD,",FAWN BLUE HOT WATER BOTTLE,FELT FARM ANIMAL CHICKEN,FELT FARM ANIMAL RABBIT,FELT FARM ANIMAL SHEEP,FELTCRAFT CHRISTMAS FAIRY,FELTCRAFT PRINCESS CHARLOTTE DOLL,FELTCRAFT PRINCESS LOLA DOLL,FELTCRAFT PRINCESS OLIVIA DOLL,FIRST AID TIN,GAOLERS KEYS DECORATIVE GARDEN,GENTLEMAN SHIRT REPAIR KIT,GLASS JAR DAISY FRESH COTTON WOOL,GLASS JAR DIGESTIVE BISCUITS,GLASS JAR ENGLISH CONFECTIONERY,GLASS JAR KINGS CHOICE,GOLD MINI TAPE MEASURE,GYMKHANA TREASURE BOOK BOX,"HOOK, 1 HANGER ,MAGIC GARDEN",HOT WATER BOTTLE TEA AND SYMPATHY,I LOVE LONDON BABY GIFT SET,I LOVE LONDON MINI BACKPACK,INFLATABLE POLITICAL GLOBE,JAM MAKING SET PRINTED,JAZZ HEARTS PURSE NOTEBOOK,JIGSAW TOADSTOOLS 3 PIECE,JIGSAW TREE WITH BIRDHOUSE,JIGSAW TREE WITH WATERING CAN,KIDS RAIN MAC BLUE,KIDS RAIN MAC PINK,LUNCH BAG CARS BLUE,LUNCH BAG DOLLY GIRL DESIGN,LUNCH BAG RED RETROSPOT,MAGIC DRAWING SLATE BAKE A CAKE,MAGIC DRAWING SLATE CIRCUS PARADE,MAGIC DRAWING SLATE LEAP FROG,MAGIC DRAWING SLATE PURDEY,MAGIC DRAWING SLATE SPACEBOY,MAGNETS PACK OF 4 SWALLOWS,MAGNETS PACK OF 4 VINTAGE COLLAGE,MINI JIGSAW BAKE A CAKE,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW DOLLY GIRL,MINI JIGSAW LEAP FROG,MINI JIGSAW PURDEY,MINI JIGSAW SPACEBOY,MINI LIGHTS WOODLAND MUSHROOMS,MINI PAINT SET VINTAGE,MINI WOODEN HAPPY BIRTHDAY GARLAND,MONEY BOX BISCUITS DESIGN,MONEY BOX CONFECTIONERY DESIGN,MONEY BOX FIRST ADE DESIGN,MONEY BOX HOUSEKEEPING DESIGN,MONEY BOX KINGS CHOICE DESIGN,MONEY BOX POCKET MONEY DESIGN,MULTICOLOUR CONFETTI IN TUBE,NURSE'S BAG SOFT TOY,PACK 20 ENGLISH ROSE PAPER NAPKINS,PACK OF 12 CIRCUS PARADE TISSUES,PACK OF 12 COLOURED PENCILS,PACK OF 12 LONDON TISSUES,PACK OF 12 RED RETROSPOT TISSUES,PACK OF 12 TRADITIONAL CRAYONS,PACK OF 12 VINTAGE DOILY TISSUES,PACK OF 20 NAPKINS PANTRY DESIGN,PACK OF 20 NAPKINS RED APPLES,PACK OF 20 SPACEBOY NAPKINS,PACK OF 6 BIRDY GIFT TAGS,PACK OF 6 HANDBAG GIFT BOXES,PACK OF 6 SWEETIE GIFT BOXES,PACK OF 60 MUSHROOM CAKE CASES,PANTRY MAGNETIC SHOPPING LIST,PANTRY SCRUBBING BRUSH,PANTRY WASHING UP BRUSH,PAPER BUNTING COLOURED LACE,PAPER BUNTING RETROSPOT,PAPER BUNTING WHITE LACE,PAPER CHAIN KIT LONDON,PARTY BUNTING,PEG BAG APPLES DESIGN,PENNY FARTHING BIRTHDAY CARD,PICTURE DOMINOES,PIGGY BANK RETROSPOT,PINK HEART CONFETTI IN TUBE,PINK BABY BUNTING,PINK FAIRY CAKE CHILDRENS APRON,PINK VINTAGE PAISLEY PICNIC BAG,PLAYING CARDS I LOVE LONDON,POLKADOT RAIN HAT,POTTING SHED TWINE,RABBIT NIGHT LIGHT,RAIN PONCHO RETROSPOT,RAINY LADIES BIRTHDAY CARD,RECIPE BOX RETROSPOT,RED BABY BUNTING,RED RETROSPOT CHARLOTTE BAG,RED RETROSPOT MINI CASES,RED RETROSPOT PEG BAG,RED RETROSPOT ROUND CAKE TINS,RED RETROSPOT STORAGE JAR,RED SPOTTY BISCUIT TIN,RED TOADSTOOL LED NIGHT LIGHT,RETROSPOT PARTY BAG + STICKER SET,RING OF ROSES BIRTHDAY CARD,ROMANTIC IMAGES GIFT WRAP SET,ROSE COTTAGE KEEPSAKE BOX,ROUND SNACK BOXES SET OF 4 FRUITS,ROUND SNACK BOXES SET OF4 WOODLAND,ROUND STORAGE TIN VINTAGE LEA

In [19]:
frequent_itemsets=apriori(basket_sets, min_support=0.07,use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [20]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.081633,(BAKING SET 9 PIECE RETROSPOT)
5,0.071429,(CHILDRENS CUTLERY DOLLY GIRL)
6,0.099490,(DOLLY GIRL LUNCH BOX)
7,0.096939,(JUMBO BAG RED RETROSPOT)
8,0.076531,(JUMBO BAG WOODLAND ANIMALS)
9,0.125000,(LUNCH BAG APPLE DESIGN)


In [21]:
#final step: generate rules with their corresponding support confidence and lift
rules= association_rules(frequent_itemsets,metric="lift",min_threshold=1)
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859,0.964734
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383,0.959283
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,0.979224
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,0.976465
4,(POSTAGE),(ALARM CLOCK BAKELIKE GREEN),0.765306,0.096939,0.084184,0.110000,1.134737,0.009996,1.014676,0.505929


In [22]:
#filter rules to check lift and confidence
rules[(rules['lift']>=6) & (rules['confidence']>=0.8)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,0.979224
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,0.976465
75,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735,0.956140
76,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122,0.981725
77,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041,0.993343
78,"(POSTAGE, ALARM CLOCK BAKELIKE GREEN)",(ALARM CLOCK BAKELIKE RED),0.084184,0.094388,0.071429,0.848485,8.989353,0.063483,5.977041,0.970454
79,"(POSTAGE, ALARM CLOCK BAKELIKE RED)",(ALARM CLOCK BAKELIKE GREEN),0.086735,0.096939,0.071429,0.823529,8.495356,0.063021,5.117347,0.966081
114,"(POSTAGE, SET/6 RED SPOTTY PAPER PLATES)",(SET/6 RED SPOTTY PAPER CUPS),0.107143,0.137755,0.102041,0.952381,6.913580,0.087281,18.107143,0.958000
115,"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS)",(SET/6 RED SPOTTY PAPER PLATES),0.117347,0.127551,0.102041,0.869565,6.817391,0.087073,6.688776,0.966763
118,(SET/6 RED SPOTTY PAPER PLATES),"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS)",0.127551,0.117347,0.102041,0.800000,6.817391,0.087073,4.413265,0.978070


**CONCLUSION:**
**-** For France looking at rules,it seems that green and red alarm clocks are purchased together and the red paper cups,napkins and plates are purchased together in a manner that it is higher than the overall probability would suggest.
**-**